In [ ]:
import os # allow for pyspark / hadoop / java instance components to be installed into the runtime
# set up a reference to the pyspark instance - stable version - 3.3.2
spark_version = 'spark-3.3.2'
# use os.environ function to reference 'SPARK_VERSION' property
os.environ['SPARK_VERSION'] = spark_version
# Install the spark and java components
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# set the runtime environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64" # sets the path to the Java Virtual Machine (Java Development Kit)
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# start a spark session by importing and using findspark
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 2s (139 kB/s)
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [ ]:
#add the file to the SparkFiles context
spark.sparkContext.addFile(url)

#make the df based on the dataset
homeDF = spark.read.option('header', 'true').csv(
    SparkFiles.get("home_sales_revised.csv"),
    inferSchema=True,
    sep=','
)
homeDF.show(truncate=False)

+------------------------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date               |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08 00:00:00|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13 00:00:00|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12 00:00:00|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16 00:00:00|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-

In [ ]:
# 2. Create a temporary view of the DataFrame.
homeDF.createOrReplaceTempView("home_sales")

In [ ]:
#Read the temporary view
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+-------

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT ROUND(AVG(price),2) FROM home_sales WHERE bedrooms=4").show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price),2) FROM home_sales WHERE bedrooms=3 AND bathrooms=3 GROUP BY date_built ").show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|            288770.3|
|      2013|           295962.27|
|      2014|           290852.27|
|      2012|           293683.19|
|      2016|           290555.07|
|      2010|           292859.62|
|      2011|           291117.47|
|      2017|           292676.79|
+----------+--------------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT date_built, ROUND(AVG(price),2) FROM home_sales WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living>=2000 GROUP BY date_built ").show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2015|           297609.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2012|           307539.97|
|      2016|            293965.1|
|      2010|           285010.22|
|      2011|           276553.81|
|      2017|           280317.58|
+----------+--------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
#What is the "view" rating for homes costing more than or equal to $350,000? Determine the run time for this query, and round off your answer to two decimal places.
start_time = time.time()

spark.sql("SELECT ROUND(AVG(view),2) FROM home_sales WHERE price>350000 ").show()

print("Execution time: %s seconds" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

Execution time: 0.8774306774139404 seconds


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(view),2) FROM home_sales WHERE price>350000 ").show()

print("Execution Time: %s seconds" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

Execution Time: 0.36176490783691406 seconds


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home_sales data
homeDF.write.partitionBy("date_built").parquet('home_sales', mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.
homeViewParquet = spark.read.parquet('home_sales')

In [ ]:
# 12. Create a temporary table for the parquet data.
homeViewParquet.createOrReplaceTempView("homeViewParquetTemp")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT ROUND(AVG(view),2) FROM homeViewParquetTemp WHERE price>350000 ").show()

print("Execution Time: %s seconds ---" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

Execution Time: 0.7150721549987793 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False